# Kevin Oakley's       BIOF509       Spring 2019      Final Project

### Predicting Bee trends from USDA census data 

In [ ]:
## Store USDA csv files in GitHub repo in the next cell

https://github.com/ContoakleyK/BIOF509-Bee-Trends.git

In [1]:
### Import packages to pull csv file from repo for analysis

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import requests
from urllib.request import urlretrieve

### This was the original query for a dataset from the USDA website.  Don't use this file!!

In [ ]:
### Assign URL to variable: url
url = 'https://raw.githubusercontent.com/ContoakleyK/BIOF509-Bee-Trends/master/USDA%20Honey%20Bee%20Colony%20Data.csv'

In [ ]:
### Apply pandas package to read the .csv file: url
df = pd.read_csv(url)

In [ ]:
print(df.shape)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
print(df.describe())

In [ ]:
print(df.info())

# Import dataset from USDA "Honey" query

In [2]:
### read in the smaller dataset of only Commodity = 'Honey' query
Honey_url = 'https://raw.githubusercontent.com/ContoakleyK/BIOF509-Bee-Trends/master/USDA%20Honey%20Data.csv'
honey_df = pd.read_csv(Honey_url)
print(honey_df.shape)

honey_df.info()

(45703, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45703 entries, 0 to 45702
Data columns (total 21 columns):
Program             45703 non-null object
Year                45703 non-null int64
Period              45703 non-null object
Week Ending         0 non-null float64
Geo Level           45703 non-null object
State               45703 non-null object
State ANSI          44952 non-null float64
Ag District         30394 non-null object
Ag District Code    30394 non-null float64
County              30394 non-null object
County ANSI         30359 non-null float64
Zip Code            0 non-null float64
Region              0 non-null float64
watershed_code      45703 non-null int64
Watershed           0 non-null float64
Commodity           45703 non-null object
Data Item           45703 non-null object
Domain              45703 non-null object
Domain Category     45703 non-null object
Value               45703 non-null object
CV (%)              14001 non-null object
dtypes: 

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (7,9,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
honey_df.head(5)

Program  Year Period  Week Ending Geo Level    State  State ANSI  \
0  CENSUS  2012   YEAR          NaN    COUNTY  ALABAMA         1.0   
1  CENSUS  2012   YEAR          NaN    COUNTY  ALABAMA         1.0   
2  CENSUS  2012   YEAR          NaN    COUNTY  ALABAMA         1.0   
3  CENSUS  2012   YEAR          NaN    COUNTY  ALABAMA         1.0   
4  CENSUS  2012   YEAR          NaN    COUNTY  ALABAMA         1.0   

  Ag District  Ag District Code   County  ...    Zip Code  Region  \
0  BLACK BELT              40.0  AUTAUGA  ...         NaN     NaN   
1  BLACK BELT              40.0  AUTAUGA  ...         NaN     NaN   
2  BLACK BELT              40.0  AUTAUGA  ...         NaN     NaN   
3  BLACK BELT              40.0   ELMORE  ...         NaN     NaN   
4  BLACK BELT              40.0   ELMORE  ...         NaN     NaN   

   watershed_code  Watershed  Commodity                           Data Item  \
0               0        NaN      HONEY  HONEY - OPERATIONS WITH PRODUCTION   
1               0        NaN      HONEY       HONEY - OPERATIONS WITH SALES   
2               0        NaN      HONEY        HONEY - SALES, MEASURED IN $   
3               0        NaN      HONEY  HONEY - OPERATIONS WITH PRODUCTION   
4               0        NaN      HONEY       HONEY - OPERATIONS WITH SALES   

  Domain Domain Category   Value CV (%)  
0  TOTAL   NOT SPECIFIED       6   33.8  
1  TOTAL   NOT SPECIFIED       5   32.9  
2  TOTAL   NOT SPECIFIED  15,000   13.6  
3  TOTAL   NOT SPECIFIED      15   33.8  
4  TOTAL   NOT SPECIFIED      13   32.9  

[5 rows x 21 columns]

In [4]:
# check for any duplicated data 
dups = honey_df.duplicated()
print(dups.describe())

# no duplicated entries

count     45703
unique        1
top       False
freq      45703
dtype: object


In [5]:
# look into the DtypeWarning for Columns (7,9,20)
dt_warn = honey_df.take([7,9,20], axis=1)
dt_warn

Ag District   County CV (%)
0                       BLACK BELT  AUTAUGA   33.8
1                       BLACK BELT  AUTAUGA   32.9
2                       BLACK BELT  AUTAUGA   13.6
3                       BLACK BELT   ELMORE   33.8
4                       BLACK BELT   ELMORE   32.9
5                       BLACK BELT   ELMORE   13.6
6                       BLACK BELT   GREENE   33.8
7                       BLACK BELT   GREENE   32.9
8                       BLACK BELT   GREENE   13.6
9                       BLACK BELT  LOWNDES   33.8
10                      BLACK BELT  LOWNDES   32.9
11                      BLACK BELT  LOWNDES    (D)
12                      BLACK BELT    MACON   33.8
13                      BLACK BELT    MACON   32.9
14                      BLACK BELT    MACON    (D)
15                      BLACK BELT  MARENGO   33.8
16                      BLACK BELT    PERRY   33.8
17                      BLACK BELT    PERRY   32.9
18                      BLACK BELT    PERRY    (D)
19                      BLACK BELT   SUMTER   33.8
20                      BLACK BELT   SUMTER   32.9
21                      BLACK BELT   SUMTER   13.6
22     COASTAL PLAINS & GULF COAST  BALDWIN   33.8
23     COASTAL PLAINS & GULF COAST  BALDWIN   32.9
24     COASTAL PLAINS & GULF COAST  BALDWIN   13.6
25     COASTAL PLAINS & GULF COAST   BUTLER   33.8
26     COASTAL PLAINS & GULF COAST   BUTLER   32.9
27     COASTAL PLAINS & GULF COAST   BUTLER   13.6
28     COASTAL PLAINS & GULF COAST  CHOCTAW   33.8
29     COASTAL PLAINS & GULF COAST  CHOCTAW   32.9
...                            ...      ...    ...
45673                          NaN      NaN    NaN
45674                          NaN      NaN    NaN
45675                          NaN      NaN    NaN
45676                          NaN      NaN    NaN
45677                          NaN      NaN    NaN
45678                          NaN      NaN    NaN
45679                          NaN      NaN    NaN
45680                          NaN      NaN    NaN
45681                          NaN      NaN    NaN
45682                          NaN      NaN    NaN
45683                          NaN      NaN    NaN
45684                          NaN      NaN    NaN
45685                          NaN      NaN    NaN
45686                          NaN      NaN    NaN
45687                          NaN      NaN    NaN
45688                          NaN      NaN    NaN
45689                          NaN      NaN    NaN
45690                          NaN      NaN    NaN
45691                          NaN      NaN    NaN
45692                          NaN      NaN    NaN
45693                          NaN      NaN    NaN
45694                          NaN      NaN    NaN
45695                          NaN      NaN    NaN
45696                          NaN      NaN    NaN
45697                          NaN      NaN    NaN
45698                          NaN      NaN    NaN
45699                          NaN      NaN    NaN
45700                          NaN      NaN    NaN
45701                          NaN      NaN    NaN
45702                          NaN      NaN    NaN

[45703 rows x 3 columns]

In [6]:
# explore these columns
print(dt_warn.describe())
print(dt_warn.info())

       Ag District      County CV (%)
count        30394       30394  14001
unique          86        1694    369
top        CENTRAL  WASHINGTON    (D)
freq          3571         356   1871
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45703 entries, 0 to 45702
Data columns (total 3 columns):
Ag District    30394 non-null object
County         30394 non-null object
CV (%)         14001 non-null object
dtypes: object(3)
memory usage: 1.0+ MB
None


In [7]:
# look for missing data in these columns
print(dt_warn.isnull().sum())

Ag District    15309
County         15309
CV (%)         31702
dtype: int64


In [8]:
# subset the dataset around these columns
hon_subset = honey_df[dt_warn.notnull()]
print(hon_subset.shape)

(45703, 21)


In [9]:
# hon is the list (np.ndarray) of column 'Commodity' to prove that the query returned only data with commodity "Honey"
hon = honey_df['Commodity'].unique()
print(hon)
print(hon.shape)

['HONEY']
(1,)


In [10]:
# lsdi is the list (np.ndarray) of column 'Data Item' to see the various sub-categories of the commodity "Honey" from my query search 
lsdi = honey_df['Data Item'].unique()
print(lsdi)
print(lsdi.shape)

['HONEY - OPERATIONS WITH PRODUCTION' 'HONEY - OPERATIONS WITH SALES'
 'HONEY - SALES, MEASURED IN $'
 'HONEY, BEE COLONIES - INVENTORY, MEASURED IN COLONIES'
 'HONEY, BEE COLONIES - OPERATIONS WITH INVENTORY'
 'HONEY, BEE COLONIES - OPERATIONS WITH SALES'
 'HONEY, BEE COLONIES - SALES, MEASURED IN COLONIES'
 'HONEY, BEE COLONIES - ADDED & REPLACED, MEASURED IN COLONIES'
 'HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DISORDER, MEASURED IN COLONIES'
 'HONEY, BEE COLONIES - LOSS, DEADOUT, MEASURED IN COLONIES'
 'HONEY, BEE COLONIES - LOSS, DEADOUT, MEASURED IN PCT OF COLONIES'
 'HONEY, BEE COLONIES, AFFECTED BY DISEASE - INVENTORY, MEASURED IN PCT OF COLONIES'
 'HONEY, BEE COLONIES, AFFECTED BY OTHER CAUSES - INVENTORY, MEASURED IN PCT OF COLONIES'
 'HONEY, BEE COLONIES, AFFECTED BY PESTICIDES - INVENTORY, MEASURED IN PCT OF COLONIES'
 'HONEY, BEE COLONIES, AFFECTED BY PESTS ((EXCL VARROA MITES)) - INVENTORY, MEASURED IN PCT OF COLONIES'
 'HONEY, BEE COLONIES, AFFECTED BY UNKNOWN CAUSES -

In [11]:
# years is the numpy.ndarray for the unique entries of year associated throughout the honey_df DataFrame
years = honey_df['Year'].unique()
print(years)
print(years.shape)

[2012 2007 2002 1997 2018 2017 2016 2015 2014 2013 2011 2010 2009 2008
 2006 2005 2004 2003 2001 2000 1999 1998 1996 1995 1994 1993 1992 1991
 1990 1989 1988 1987]
(32,)


In [12]:
# period is the np.ndarray of time designations for entries
period = honey_df['Period'].unique()
print(period)
print(period.shape)

['YEAR' 'END OF DEC' 'FIRST OF JAN' 'JAN THRU MAR' 'APR THRU JUN'
 'FIRST OF APR' 'MARKETING YEAR' 'FIRST OF JUL' 'JUL THRU SEP'
 'FIRST OF OCT' 'OCT THRU DEC' 'MID DEC']
(12,)


In [13]:
# census is the np.ndarray for the 'census' and 'survey' designations of the 'Program' column
census = honey_df['Program'].unique()
print(census)
print(census.shape)

['CENSUS' 'SURVEY']
(2,)


In [ ]:
# Use Pandas to encode us values as 1 and others as 0
honey_df['cs_enc'] = honey_df['Program'].apply(lambda val: 1 if val == 'CENSUS' else 0)

# Print the number of unique type values
print(len(honey_df['CENSUS'].unique()))

# Create a one-hot encoded set of the Domain values
cen_sur_set = pd.get_dummies(honey_df['Program'])

# Concatenate this set back to the ufo DataFrame
honey_df = pd.concat([honey_df, cen_sur_set], axis=1)

In [14]:
# state is the numpy.ndarray for the States represented throughout the honey_df DataFrame.  Notice entries for "US TOTAL" and "OTHER STATES"
state = honey_df['State'].unique()
print(state)
print(state.shape)

['ALABAMA' 'ALASKA' 'ARIZONA' 'ARKANSAS' 'CALIFORNIA' 'COLORADO'
 'CONNECTICUT' 'DELAWARE' 'FLORIDA' 'GEORGIA' 'HAWAII' 'IDAHO' 'ILLINOIS'
 'INDIANA' 'IOWA' 'KANSAS' 'KENTUCKY' 'LOUISIANA' 'MAINE' 'MARYLAND'
 'MASSACHUSETTS' 'MICHIGAN' 'MINNESOTA' 'MISSISSIPPI' 'MISSOURI' 'MONTANA'
 'NEBRASKA' 'NEVADA' 'NEW HAMPSHIRE' 'NEW JERSEY' 'NEW MEXICO' 'NEW YORK'
 'NORTH CAROLINA' 'NORTH DAKOTA' 'OHIO' 'OKLAHOMA' 'OREGON' 'PENNSYLVANIA'
 'RHODE ISLAND' 'SOUTH CAROLINA' 'SOUTH DAKOTA' 'TENNESSEE' 'TEXAS' 'UTAH'
 'VERMONT' 'VIRGINIA' 'WASHINGTON' 'WEST VIRGINIA' 'WISCONSIN' 'WYOMING'
 'US TOTAL' 'OTHER STATES']
(52,)


In [15]:
# sansi is the numpy.ndarray for the American National Standards Institute. ANSI codes are a standardized set of numeric or alphabetic codes issued by the American National Standards Institute (ANSI) to ensure uniform identification of geographic entities through all federal government agencies. These standards replace the Federal Information Processing Standards (FIPS) codes previously issued by the National Institute of Standards and Technology (NIST). See http://www.census.gov/geo/www/ansi/ for more information.
sansi = honey_df['State ANSI'].unique()
print(sansi)
print(sansi.shape)

[ 1.  2.  4.  5.  6.  8.  9. 10. 12. 13. 15. 16. 17. 18. 19. 20. 21. 22.
 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39. 40.
 41. 42. 44. 45. 46. 47. 48. 49. 50. 51. 53. 54. 55. 56. nan]
(51,)


In [16]:
# ag_dist is the numpy.ndarray for the Agricultural Districts represented throughout the honey_df DataFrame.  
ag_dist = honey_df['Ag District'].unique()
print(ag_dist)
print(ag_dist.shape)

['BLACK BELT' 'COASTAL PLAINS & GULF COAST' 'MOUNTAINS & EASTERN VALLEY'
 'NORTHERN VALLEY' 'UPPER PLAINS & PIEDMONT' 'WIREGRASS' 'KENAI PENINSULA'
 'MATANUSKA-SUSITNA-ANCHORAGE' 'TANANA VALLEY' 'NORTHERN' 'SOUTHERN'
 'CENTRAL' 'EAST CENTRAL' 'NORTH CENTRAL' 'NORTHEAST' 'NORTHWEST'
 'SOUTH CENTRAL' 'SOUTHEAST' 'SOUTHWEST' 'WEST CENTRAL' 'CENTRAL COAST'
 'NORTHERN COAST' 'SACRAMENTO VALLEY' 'SAN JOAQUIN VALLEY'
 'SIERRA MOUNTAINS' 'SISKIYOU-SHASTA' 'SOUTHERN CALIFORNIA'
 'NORTHWEST AND MOUNTAIN' 'SAN LUIS VALLEY' 'ALL COUNTIES' 'HAWAII ISLAND'
 'KAUAI AND NIIHAU ISLANDS' 'MAUI, MOLOKAI & LANAI ISLAND' 'OAHU ISLAND'
 'EAST' 'NORTH' 'EAST SOUTHEAST' 'WEST' 'WEST SOUTHWEST' 'BLUEGRASS'
 'EASTERN OR MOUNTAIN' 'MIDWESTERN' 'PURCHASE' 'SOUTH'
 'LOWER EASTERN SHORE' 'UPPER EASTERN SHORE' 'WESTERN' 'UPPER PENINSULA'
 'LOWER DELTA' 'SOUTHEAST AND COASTAL' 'UPPER DELTA' 'EASTERN'
 'LONG ISLAND' 'CENTRAL COASTAL' 'CENTRAL PIEDMONT' 'NORTHERN COASTAL'
 'NORTHERN MOUNTAIN' 'NORTHERN PIEDMONT' 'SOUTH

In [17]:
# ag_dist_cd is the numpy.ndarray for the Agricultural District Codes represented throughout the honey_df DataFrame.  
ag_dist_cd = honey_df['Ag District Code'].unique()
print(ag_dist_cd)
print(ag_dist_cd.shape)

[40. 50. 20. 10. 30. 60. 80. 90. 70. 51. 11. 24. 35. 62. 91. 82. 52. 97.
 21. 81. 96. 22. nan 12.]
(24,)


In [18]:
# county is the numpy.ndarray for the Counties represented throughout the honey_df DataFrame.
county = honey_df['County'].unique()
print(county)
print(county.shape)

['AUTAUGA' 'ELMORE' 'GREENE' ... 'HANSFORD' 'LAMB' 'PECOS']
(1695,)


In [19]:
# cansi is the numpy.ndarray for the Countie ANSI codes represented throughout the honey_df DataFrame.
cansi = honey_df['County ANSI'].unique()
print(cansi)
print(cansi.shape)

[  1.  51.  63.  85.  87.  91. 105. 119.   3.  13.  23.  25.  35.  53.
  97.  99. 129.   9.  15.  19.  29.  43.  49.  55.  71.  95. 115.  33.
  59.  77.  79.  83.  89.  93. 103. 133.   7.  17.  21.  27.  37.  57.
  73.  75.  81. 107. 111. 117. 121. 123. 125. 127.   5.  31.  39.  41.
  45.  61.  67.  69. 109. 113.  nan  12. 147.  65. 135. 137. 141. 145.
 143.  47. 131. 149.  11. 101.  86. 153. 159. 167. 169. 175. 207. 209.
 211. 217. 225. 235. 237. 247. 265. 163. 189. 251. 139. 157. 187. 219.
 227. 281. 291. 297. 311. 181. 195. 221. 241. 257. 317. 213. 223. 233.
 295. 313. 155. 161. 173. 185. 271. 277. 287. 321. 179. 183. 229. 267.
 299. 305. 177. 201. 205. 243. 253. 261. 273. 151. 171. 197. 199. 231.
 255. 263. 269. 285. 293. 203. 191. 193. 165. 215. 239. 186. 510. 309.
 331. 349. 397. 439. 491. 409. 333. 337. 363. 367. 425. 497. 315. 343.
 347. 365. 379. 387. 401. 419. 423. 449. 459. 467. 499. 289. 339. 351.
 373. 395. 403. 407. 455. 457. 471. 473. 411. 451. 463. 485. 325. 453.
 477. 

In [20]:
# wash_cd is the numpy.ndarray for the Watershed Code represented throughout the honey_df DataFrame. Seems uninformative
wash_cd = honey_df['watershed_code'].unique()
print(wash_cd)
print(wash_cd.shape)

[0]
(1,)


In [21]:
# domain is the list (np.ndarray) of column 'Domain' to see the various sub-categories
domain = honey_df['Domain'].unique()
print(domain)
print(domain.shape)

['TOTAL' 'OPERATORS' 'OPERATORS, PRINCIPAL' 'INVENTORY OF BEE COLONIES']
(4,)


In [22]:
# get more information about the 4 different labels in the Domain column
print(honey_df['Domain'].describe())  # This shows that 44853 of the 45703 entries are 'TOTAL'

inv_beeC = honey_df['Domain'] == 'INVENTORY OF BEE COLONIES'
print(inv_beeC.describe())  # this tells me that only 102 entries are True for 'INVENTORY OF BEE COLONIES'

oper = honey_df['Domain'] == 'OPERATORS'
print(oper.describe())  # this tells me that only 454 entries are True for 'OPERATORS'

oper_prin = honey_df['Domain'] == 'OPERATORS, PRINCIPAL'
print(oper_prin.describe())  # this tells me that only 294 entries are True for 'OPERATORS, PRINCIPAL'

count     45703
unique        4
top       TOTAL
freq      44853
Name: Domain, dtype: object
count     45703
unique        2
top       False
freq      45601
Name: Domain, dtype: object
count     45703
unique        2
top       False
freq      45249
Name: Domain, dtype: object
count     45703
unique        2
top       False
freq      45409
Name: Domain, dtype: object


In [23]:
# try encoding the labels of the Domain column with scikit-learn's LabelEncoder method to do that transformation.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(honey_df['Domain'])

list(le.classes_)

le.transform(['INVENTORY OF BEE COLONIES', 'OPERATORS', 'OPERATORS, PRINCIPAL', 'TOTAL']) 


array([0, 1, 2, 3])

In [24]:
# fit and transform the previous cell and compare to the original column
honey_df['Domain_enc'] = le.fit_transform(honey_df['Domain'])
print(honey_df[['Domain_enc','Domain']].head())

# explore this new column
print(honey_df.shape)

print(honey_df[['Domain_enc']].describe())
print(honey_df[['Domain_enc']].info())


   Domain_enc Domain
0           3  TOTAL
1           3  TOTAL
2           3  TOTAL
3           3  TOTAL
4           3  TOTAL
(45703, 22)
         Domain_enc
count  45703.000000
mean       2.967004
std        0.255277
min        0.000000
25%        3.000000
50%        3.000000
75%        3.000000
max        3.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45703 entries, 0 to 45702
Data columns (total 1 columns):
Domain_enc    45703 non-null int64
dtypes: int64(1)
memory usage: 357.1 KB
None


In [25]:
# just curious, drop all rows not equal to 'INVENTORY OF BEE COLONIES'.  See what data this leaves you to play with.
# step 1 - create boolean Series
criteria = honey_df['Domain'] == 'INVENTORY OF BEE COLONIES'
# step 2 - do boolean selection
honey_df[criteria]

Program  Year          Period  Week Ending Geo Level     State  \
32695  SURVEY  2017  MARKETING YEAR          NaN  NATIONAL  US TOTAL   
32697  SURVEY  2017  MARKETING YEAR          NaN  NATIONAL  US TOTAL   
32700  SURVEY  2017  MARKETING YEAR          NaN  NATIONAL  US TOTAL   
32827  SURVEY  2017            YEAR          NaN  NATIONAL  US TOTAL   
32828  SURVEY  2017            YEAR          NaN  NATIONAL  US TOTAL   
32829  SURVEY  2017            YEAR          NaN  NATIONAL  US TOTAL   
32830  SURVEY  2017            YEAR          NaN  NATIONAL  US TOTAL   
32831  SURVEY  2017            YEAR          NaN  NATIONAL  US TOTAL   
32832  SURVEY  2017            YEAR          NaN  NATIONAL  US TOTAL   
32833  SURVEY  2017            YEAR          NaN  NATIONAL  US TOTAL   
32834  SURVEY  2017    FIRST OF JAN          NaN  NATIONAL  US TOTAL   
32882  SURVEY  2017    JAN THRU MAR          NaN  NATIONAL  US TOTAL   
32885  SURVEY  2017    JAN THRU MAR          NaN  NATIONAL  US TOTAL   
32887  SURVEY  2017    JAN THRU MAR          NaN  NATIONAL  US TOTAL   
32895  SURVEY  2017    JAN THRU MAR          NaN  NATIONAL  US TOTAL   
32897  SURVEY  2017    JAN THRU MAR          NaN  NATIONAL  US TOTAL   
33451  SURVEY  2017    APR THRU JUN          NaN  NATIONAL  US TOTAL   
33454  SURVEY  2017    APR THRU JUN          NaN  NATIONAL  US TOTAL   
33456  SURVEY  2017    APR THRU JUN          NaN  NATIONAL  US TOTAL   
33464  SURVEY  2017    APR THRU JUN          NaN  NATIONAL  US TOTAL   
33466  SURVEY  2017    APR THRU JUN          NaN  NATIONAL  US TOTAL   
34020  SURVEY  2017    FIRST OF APR          NaN  NATIONAL  US TOTAL   
34068  SURVEY  2017    FIRST OF JUL          NaN  NATIONAL  US TOTAL   
34116  SURVEY  2017    JUL THRU SEP          NaN  NATIONAL  US TOTAL   
34119  SURVEY  2017    JUL THRU SEP          NaN  NATIONAL  US TOTAL   
34121  SURVEY  2017    JUL THRU SEP          NaN  NATIONAL  US TOTAL   
34129  SURVEY  2017    JUL THRU SEP          NaN  NATIONAL  US TOTAL   
34131  SURVEY  2017    JUL THRU SEP          NaN  NATIONAL  US TOTAL   
34685  SURVEY  2017    FIRST OF OCT          NaN  NATIONAL  US TOTAL   
34733  SURVEY  2017    OCT THRU DEC          NaN  NATIONAL  US TOTAL   
...       ...   ...             ...          ...       ...       ...   
38123  SURVEY  2015            YEAR          NaN  NATIONAL  US TOTAL   
38124  SURVEY  2015            YEAR          NaN  NATIONAL  US TOTAL   
38125  SURVEY  2015            YEAR          NaN  NATIONAL  US TOTAL   
38126  SURVEY  2015            YEAR          NaN  NATIONAL  US TOTAL   
38127  SURVEY  2015            YEAR          NaN  NATIONAL  US TOTAL   
38128  SURVEY  2015            YEAR          NaN  NATIONAL  US TOTAL   
38129  SURVEY  2015    FIRST OF JAN          NaN  NATIONAL  US TOTAL   
38177  SURVEY  2015    JAN THRU MAR          NaN  NATIONAL  US TOTAL   
38180  SURVEY  2015    JAN THRU MAR          NaN  NATIONAL  US TOTAL   
38182  SURVEY  2015    JAN THRU MAR          NaN  NATIONAL  US TOTAL   
38190  SURVEY  2015    JAN THRU MAR          NaN  NATIONAL  US TOTAL   
38192  SURVEY  2015    JAN THRU MAR          NaN  NATIONAL  US TOTAL   
38746  SURVEY  2015    APR THRU JUN          NaN  NATIONAL  US TOTAL   
38749  SURVEY  2015    APR THRU JUN          NaN  NATIONAL  US TOTAL   
38751  SURVEY  2015    APR THRU JUN          NaN  NATIONAL  US TOTAL   
38759  SURVEY  2015    APR THRU JUN          NaN  NATIONAL  US TOTAL   
38761  SURVEY  2015    APR THRU JUN          NaN  NATIONAL  US TOTAL   
39315  SURVEY  2015    FIRST OF APR          NaN  NATIONAL  US TOTAL   
39363  SURVEY  2015    FIRST OF JUL          NaN  NATIONAL  US TOTAL   
39411  SURVEY  2015    JUL THRU SEP          NaN  NATIONAL  US TOTAL   
39414  SURVEY  2015    JUL THRU SEP          NaN  NATIONAL  US TOTAL   
39416  SURVEY  2015    JUL THRU SEP          NaN  NATIONAL  US TOTAL   
39424  SURVEY  2015    JUL THRU SEP          NaN  NATIONAL  US TOTAL   
39426  SURVEY  2015    JUL THRU SEP          N

In [26]:
# looks like the dataset consists of only [State]='US TOTAL' from 2015 & 2017 that have the same 'Domain' & 'Domain Category' identities. The 'Data Item' and 'Value' columns vary
bcol_dead = honey_df['Data Item'] == 'HONEY, BEE COLONIES - LOSS, DEADOUT, MEASURED IN COLONIES'
bcol_dead.describe()


count     45703
unique        2
top       False
freq      45033
Name: Data Item, dtype: object

In [27]:
# value is the list (np.ndarray) of column 'Value' to see the various sub-categories
value = honey_df['Value'].unique()
print(value)
print(value.shape)

['6' '5' '15,000' ... '1,188,000' '531,000' '5,574,000']
(4542,)


# Preprocessing Data

In [28]:
# Start Preprocessing: Look through the honey_df for missing data in columns of interest based on the above exploration
print(honey_df.dtypes)
print(honey_df.shape)

honey_drop = honey_df.dropna(axis=1, thresh=3)
print(honey_drop.info())

Program              object
Year                  int64
Period               object
Week Ending         float64
Geo Level            object
State                object
State ANSI          float64
Ag District          object
Ag District Code    float64
County               object
County ANSI         float64
Zip Code            float64
Region              float64
watershed_code        int64
Watershed           float64
Commodity            object
Data Item            object
Domain               object
Domain Category      object
Value                object
CV (%)               object
Domain_enc            int64
dtype: object
(45703, 22)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45703 entries, 0 to 45702
Data columns (total 18 columns):
Program             45703 non-null object
Year                45703 non-null int64
Period              45703 non-null object
Geo Level           45703 non-null object
State               45703 non-null object
State ANSI          44952 non-null float

In [29]:
from sklearn.model_selection import train_test_split

# Stratified Sampling using the 'Data Item' column
# Create a data with all columns except Data Item
d_item_X = honey_df.drop("Data Item", axis=1)

# Create a Data Item labels dataset
d_item_y = honey_df[["Data Item"]]

# Use stratified sampling to split up the dataset according to the d_item_y dataset
X_train, X_test, y_train, y_test = train_test_split(d_item_X, d_item_y, stratify=d_item_y)

# Print out the Data Item counts on the training y labels
print(y_train["Data Item"].value_counts())


HONEY, BEE COLONIES - INVENTORY, MEASURED IN COLONIES                                                    7848
HONEY, BEE COLONIES - OPERATIONS WITH INVENTORY                                                          6306
HONEY - OPERATIONS WITH PRODUCTION                                                                       5476
HONEY - OPERATIONS WITH SALES                                                                            1877
HONEY - SALES, MEASURED IN $                                                                             1877
HONEY - PRODUCTION, MEASURED IN LB / COLONY                                                              1042
HONEY - PRODUCTION, MEASURED IN $                                                                        1039
HONEY - STOCKS, MEASURED IN LB                                                                           1039
HONEY, BEE COLONIES - SALES, MEASURED IN COLONIES                                                         883
HONEY, BEE

In [30]:
# Check how many values are missing in the Ag District Code column
print(honey_df['Ag District Code'].isnull().sum())

# Subset the honey dataset
honey_subset = honey_drop[honey_drop['Ag District Code'].notnull()]

# Print out the shape of the subset
print(honey_subset.shape)


15309
(30394, 18)


In [31]:
# I got a warning in the honey_df.info() output for the mix of dtypes in 'Ag District Code' column.  Looking into the column earlier I saw that there are 24 unique codes, one of which is 'nan'.
# Check how many values are missing in the 'Ag District Code' column
print(honey_subset[['Ag District Code']].isnull().sum())

# Keep only rows where 'Ag District Code' are not null
honey_subset_no_missing = honey_subset[honey_subset["Ag District Code"].notnull()]

# Print out the shape of the new dataset
print(honey_subset_no_missing.shape)
print(honey_subset_no_missing.info())

Ag District Code    0
dtype: int64
(30394, 18)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30394 entries, 0 to 31120
Data columns (total 18 columns):
Program             30394 non-null object
Year                30394 non-null int64
Period              30394 non-null object
Geo Level           30394 non-null object
State               30394 non-null object
State ANSI          30394 non-null float64
Ag District         30394 non-null object
Ag District Code    30394 non-null float64
County              30394 non-null object
County ANSI         30359 non-null float64
watershed_code      30394 non-null int64
Commodity           30394 non-null object
Data Item           30394 non-null object
Domain              30394 non-null object
Domain Category     30394 non-null object
Value               30394 non-null object
CV (%)              13672 non-null object
Domain_enc          30394 non-null int64
dtypes: float64(3), int64(3), object(12)
memory usage: 4.4+ MB
None


In [32]:
# I got a warning in the honey_df.info() output for the mix of dtypes in 'Ag District Code' column.  Looking into the column earlier I saw that there are 24 unique codes, one of which is 'nan'.
# Check how many values are missing in the 'Ag District Code' column
print(honey_subset[['State ANSI']].isnull().sum())

# Keep only rows where 'State ANSI' are not null
honey_subset_clean_ANSI = honey_subset[honey_subset['State ANSI'].notnull()]

# Print out the shape of the new dataset
print(honey_subset_clean_ANSI.shape)
print(honey_subset_clean_ANSI.info())

State ANSI    0
dtype: int64
(30394, 18)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30394 entries, 0 to 31120
Data columns (total 18 columns):
Program             30394 non-null object
Year                30394 non-null int64
Period              30394 non-null object
Geo Level           30394 non-null object
State               30394 non-null object
State ANSI          30394 non-null float64
Ag District         30394 non-null object
Ag District Code    30394 non-null float64
County              30394 non-null object
County ANSI         30359 non-null float64
watershed_code      30394 non-null int64
Commodity           30394 non-null object
Data Item           30394 non-null object
Domain              30394 non-null object
Domain Category     30394 non-null object
Value               30394 non-null object
CV (%)              13672 non-null object
Domain_enc          30394 non-null int64
dtypes: float64(3), int64(3), object(12)
memory usage: 4.4+ MB
None


In [33]:
# Change the 'Value' column entries to float64 dtypes from object dtypes
honey_subset_no_missing["Value"] = honey_subset_no_missing["Value"].astype(float)
# Check the column types
print(honey_subset_no_missing["Value"].dtypes)

# looking into stackoverflow to solve the ValueError I found this?  frame["time"]=frame["time"].apply(lambda x:x[0])
# but when I applied it to the current column it gave object dtype.
honey_subset_no_missing["Value"].head()

ValueError: could not convert string to float: '15,000'

# Standardizing the Data


In [34]:
# Check the variance of the data
print(honey_df.var())

Year                  39.876983
Week Ending                 NaN
State ANSI           232.062794
Ag District Code     657.688951
County ANSI         6684.426294
Zip Code                    NaN
Region                      NaN
watershed_code         0.000000
Watershed                   NaN
Domain_enc             0.065167
dtype: float64


In [35]:
# Check the variance of the dropped data 
print(honey_drop.var())

Year                  39.876983
State ANSI           232.062794
Ag District Code     657.688951
County ANSI         6684.426294
watershed_code         0.000000
Domain_enc             0.065167
dtype: float64


In [36]:
# Import Train/Test Split + Fit/Predict/Accuracy classification tools
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split
knn = KNeighborsClassifier()

In [37]:
# drop NaN values from the 'State ANSI' column before stratified sampling
sansi_cln = honey_drop['State ANSI'].dropna()
print(sansi_cln.shape)
print(honey_drop.shape)
sansi_cln.describe()


(44952,)
(45703, 18)


count    44952.000000
mean        30.157234
std         15.233607
min          1.000000
25%         18.000000
50%         29.000000
75%         42.000000
max         56.000000
Name: State ANSI, dtype: float64

In [38]:
# Create a data with all columns except 'State ANSI'
sansi_X = honey_subset_no_missing.drop('State ANSI', axis=1)

# Create a 'State ANSI' labels dataset
sansi_y = honey_subset_no_missing[['State ANSI']]

# Use stratified sampling to split up the dataset according to the sansi_y dataset
X_train, X_test, y_train, y_test = train_test_split(sansi_X, sansi_y, stratify=sansi_y)

# Print out the category_desc counts on the training y labels
print(y_train['State ANSI'].value_counts())
print('\n', y_train.head())
sansi_y.(fe_data)

SyntaxError: invalid syntax (<ipython-input-38-deca4cf4c527>, line 13)

In [39]:
# investigate the columns of interest
print('\n Data Item Column')
print(honey_subset_no_missing[['Data Item']].describe())  


 Data Item Column
                                                Data Item
count                                               30394
unique                                                  7
top     HONEY, BEE COLONIES - INVENTORY, MEASURED IN C...
freq                                                 8167


In [40]:
# reminder of the variances from before cleaning
print(honey_drop.var())

# Apply the log normalization function to the State ANSI, Ag District Code and County ANSI columns
honey_df['State_ANSI_log'] = np.log(honey_df['State ANSI'])
honey_df['Ag_Dist_Code_log'] = np.log(honey_df['Ag District Code'])
honey_df['CANSI_log'] = np.log(honey_df['County ANSI'])


# Check the variance of the columns again
print(honey_df['State_ANSI_log'].var())
print(honey_df['Ag_Dist_Code_log'].var())
print(honey_df['CANSI_log'].var())

Year                  39.876983
State ANSI           232.062794
Ag District Code     657.688951
County ANSI         6684.426294
watershed_code         0.000000
Domain_enc             0.065167
dtype: float64
0.6364524975968708
0.4839118243552205
1.3440957842382475


In [41]:
# now the variances after cleaning
print(honey_subset_no_missing.var())

# Apply the log normalization function to the State ANSI, Ag District Code and County ANSI columns
honey_subset_no_missing['State_ANSI_log'] = np.log(honey_subset_no_missing['State ANSI'])
honey_subset_no_missing['Ag_Dist_Code_log'] = np.log(honey_subset_no_missing['Ag District Code'])
honey_subset_no_missing['CANSI_log'] = np.log(honey_subset_no_missing['County ANSI'])


# Check the variance of the columns again
print(honey_subset_no_missing['State_ANSI_log'].var())
print(honey_subset_no_missing['Ag_Dist_Code_log'].var())
print(honey_subset_no_missing['CANSI_log'].var())

honey_subset_no_missing.shape

Year                  16.937728
State ANSI           227.505930
Ag District Code     657.688951
County ANSI         6684.426294
watershed_code         0.000000
Domain_enc             0.059254
dtype: float64
0.6215499608070967
0.4839118243552205
1.3440957842382475


(30394, 21)

In [42]:
honey_subset_no_missing

Program  Year      Period Geo Level    State  State ANSI  \
0      CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
1      CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
2      CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
3      CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
4      CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
5      CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
6      CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
7      CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
8      CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
9      CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
10     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
11     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
12     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
13     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
14     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
15     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
16     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
17     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
18     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
19     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
20     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
21     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
22     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
23     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
24     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
25     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
26     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
27     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
28     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
29     CENSUS  2012        YEAR    COUNTY  ALABAMA         1.0   
...       ...   ...         ...       ...      ...         ...   
31091  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31092  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31093  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31094  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31095  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31096  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31097  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31098  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31099  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31100  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31101  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31102  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31103  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31104  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31105  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31106  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31107  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31108  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31109  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31110  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31111  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31112  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31113  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31114  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31115  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31116  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31117  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31118  CENSUS  2002  END OF DEC    COUNTY  WYOMING        56.0   
31119  CENSUS  2002  END OF DEC    COUNTY  WYO

In [43]:
# look at accuracy of k-nearest neighbors (knn) model on the set using the normalized Year

# Create a data with all columns except 'Year'
yr_X = honey_subset_no_missing.drop('Year', axis=1)

# Create a 'Year' labels dataset
yr_y = honey_subset_no_missing[['Year']]

# Use stratified sampling to split up the dataset according to the yr_y dataset
X_train, X_test, y_train, y_test = train_test_split(yr_X, yr_y, stratify=yr_y)

# Fit the k-nearest neighbors model to the training data
knn.fit(X_train, y_train)

# Score the model on the test data
print(knn.score(X_test, y_test))


# Print out the category_desc counts on the training y labels
print(y_train['Year'].value_counts())
print('\n', y_train.head())

ValueError: could not convert string to float: 'CENSUS'

# Feature Engineering - from sklearn

In [44]:
# pull features from above cleaned data to use for feature engineering
fe_data = honey_subset_no_missing.take([1,12,17,18,19,20], axis=1)
print(fe_data.head())
print(fe_data.describe())

   Year                           Data Item  Domain_enc  State_ANSI_log  \
0  2012  HONEY - OPERATIONS WITH PRODUCTION           3             0.0   
1  2012       HONEY - OPERATIONS WITH SALES           3             0.0   
2  2012        HONEY - SALES, MEASURED IN $           3             0.0   
3  2012  HONEY - OPERATIONS WITH PRODUCTION           3             0.0   
4  2012       HONEY - OPERATIONS WITH SALES           3             0.0   

   Ag_Dist_Code_log  CANSI_log  
0          3.688879   0.000000  
1          3.688879   0.000000  
2          3.688879   0.000000  
3          3.688879   3.931826  
4          3.688879   3.931826  
               Year    Domain_enc  State_ANSI_log  Ag_Dist_Code_log  \
count  30394.000000  30394.000000    30394.000000      30394.000000   
mean    2007.935711      2.965059        3.201787          3.678845   
std        4.115547      0.243420        0.788384          0.695638   
min     2002.000000      1.000000        0.000000          2.302585

In [45]:
# Text features from the 'Data Item' column
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
X_d = vec.fit_transform(fe_data['Data Item'])
X_d

<30394x10 sparse matrix of type '<class 'numpy.int64'>'
	with 158360 stored elements in Compressed Sparse Row format>

In [46]:
# convert to a DataFrame with labeled columns
pd.DataFrame(X_d.toarray(), columns=vec.get_feature_names())


bee  colonies  honey  in  inventory  measured  operations  production  \
0        0         0      1   0          0         0           1           1   
1        0         0      1   0          0         0           1           0   
2        0         0      1   1          0         1           0           0   
3        0         0      1   0          0         0           1           1   
4        0         0      1   0          0         0           1           0   
5        0         0      1   1          0         1           0           0   
6        0         0      1   0          0         0           1           1   
7        0         0      1   0          0         0           1           0   
8        0         0      1   1          0         1           0           0   
9        0         0      1   0          0         0           1           1   
10       0         0      1   0          0         0           1           0   
11       0         0      1   1          0         1           0           0   
12       0         0      1   0          0         0           1           1   
13       0         0      1   0          0         0           1           0   
14       0         0      1   1          0         1           0           0   
15       0         0      1   0          0         0           1           1   
16       0         0      1   0          0         0           1           1   
17       0         0      1   0          0         0           1           0   
18       0         0      1   1          0         1           0           0   
19       0         0      1   0          0         0           1           1   
20       0         0      1   0          0         0           1           0   
21       0         0      1   1          0         1           0           0   
22       0         0      1   0          0         0           1           1   
23       0         0      1   0          0         0           1           0   
24       0         0      1   1          0         1           0           0   
25       0         0      1   0          0         0           1           1   
26       0         0      1   0          0         0           1           0   
27       0         0      1   1          0         1           0           0   
28       0         0      1   0          0         0           1           1   
29       0         0      1   0          0         0           1           0   
...    ...       ...    ...  ..        ...       ...         ...         ...   
30364    1         2      1   1          1         1           0           0   
30365    1         1      1   0          1         0           1           0   
30366    1         2      1   1          1         1           0           0   
30367    1         1      1   0          1         0           1           0   
30368    1         2      1   1          1         1           0           0   
30369    1         1      1   0          1         0           1           0   
30370    1         2      1   1          1         1           0           0   
30371    1         1      1   0          1         0           1           0   
30372    1         2      1   1          1         1           0           0   
30373    1         1      1   0          1         0           1           0   
30374    1         2      1   1          1         1           0           0   
30375    1         1      1   0          1         0           1           0   
30376    1         2      1   1          1         1           0           0   
30377    1         1      1   0          1         0           1           0   
30378    1         2      1   1          1         1           0           0   
30379    1         1      1   0          1         0           1           0   
30380    1         2      1   1          1         1           0           0   
30381    1         1      1   0          1         0           1           0   
30382  

In [47]:
# Weight the word counts by how often they appear using term frequency-inverse document frequency (tf-idf)
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
X_d = vec.fit_transform(fe_data['Data Item'])
pd.DataFrame(X_d.toarray(), columns=vec.get_feature_names())

bee  colonies     honey        in  inventory  measured  \
0      0.000000  0.000000  0.295867  0.000000   0.000000  0.000000   
1      0.000000  0.000000  0.294640  0.000000   0.000000  0.000000   
2      0.000000  0.000000  0.259983  0.509322   0.000000  0.509322   
3      0.000000  0.000000  0.295867  0.000000   0.000000  0.000000   
4      0.000000  0.000000  0.294640  0.000000   0.000000  0.000000   
5      0.000000  0.000000  0.259983  0.509322   0.000000  0.509322   
6      0.000000  0.000000  0.295867  0.000000   0.000000  0.000000   
7      0.000000  0.000000  0.294640  0.000000   0.000000  0.000000   
8      0.000000  0.000000  0.259983  0.509322   0.000000  0.509322   
9      0.000000  0.000000  0.295867  0.000000   0.000000  0.000000   
10     0.000000  0.000000  0.294640  0.000000   0.000000  0.000000   
11     0.000000  0.000000  0.259983  0.509322   0.000000  0.509322   
12     0.000000  0.000000  0.295867  0.000000   0.000000  0.000000   
13     0.000000  0.000000  0.294640  0.000000   0.000000  0.000000   
14     0.000000  0.000000  0.259983  0.509322   0.000000  0.509322   
15     0.000000  0.000000  0.295867  0.000000   0.000000  0.000000   
16     0.000000  0.000000  0.295867  0.000000   0.000000  0.000000   
17     0.000000  0.000000  0.294640  0.000000   0.000000  0.000000   
18     0.000000  0.000000  0.259983  0.509322   0.000000  0.509322   
19     0.000000  0.000000  0.295867  0.000000   0.000000  0.000000   
20     0.000000  0.000000  0.294640  0.000000   0.000000  0.000000   
21     0.000000  0.000000  0.259983  0.509322   0.000000  0.509322   
22     0.000000  0.000000  0.295867  0.000000   0.000000  0.000000   
23     0.000000  0.000000  0.294640  0.000000   0.000000  0.000000   
24     0.000000  0.000000  0.259983  0.509322   0.000000  0.509322   
25     0.000000  0.000000  0.295867  0.000000   0.000000  0.000000   
26     0.000000  0.000000  0.294640  0.000000   0.000000  0.000000   
27     0.000000  0.000000  0.259983  0.509322   0.000000  0.509322   
28     0.000000  0.000000  0.295867  0.000000   0.000000  0.000000   
29     0.000000  0.000000  0.294640  0.000000   0.000000  0.000000   
...         ...       ...       ...       ...        ...       ...   
30364  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30365  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30366  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30367  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30368  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30369  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30370  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30371  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30372  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30373  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30374  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30375  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30376  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30377  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30378  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30379  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30380  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30381  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30382  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30383  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30384  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30385  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30386  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30387  0.424215  0.424215  0.282375  0.000000   0.457721  0.000000   
30388  0.316097  0.632194  0.210407  0.412200   0.341064  0.412200   
30389  0.424215  0.424

In [64]:
# pull features from above cleaned data ** WITHOUT 'Data Item' column ** to use for feature engineering
fe2_data = honey_subset_no_missing.take([1,17,18,19,20], axis=1)

print(fe2_data.shape)
print(fe2_data.head())
print(fe2_data.describe())

f2_drop = fe2_data.dropna()
f2_drop

(30394, 5)
   Year  Domain_enc  State_ANSI_log  Ag_Dist_Code_log  CANSI_log
0  2012           3             0.0          3.688879   0.000000
1  2012           3             0.0          3.688879   0.000000
2  2012           3             0.0          3.688879   0.000000
3  2012           3             0.0          3.688879   3.931826
4  2012           3             0.0          3.688879   3.931826
               Year    Domain_enc  State_ANSI_log  Ag_Dist_Code_log  \
count  30394.000000  30394.000000    30394.000000      30394.000000   
mean    2007.935711      2.965059        3.201787          3.678845   
std        4.115547      0.243420        0.788384          0.695638   
min     2002.000000      1.000000        0.000000          2.302585   
25%     2002.000000      3.000000        2.890372          3.401197   
50%     2007.000000      3.000000        3.367296          3.912023   
75%     2012.000000      3.000000        3.806662          4.248495   
max     2012.000000      3.0000

Year  Domain_enc  State_ANSI_log  Ag_Dist_Code_log  CANSI_log
0      2012           3        0.000000          3.688879   0.000000
1      2012           3        0.000000          3.688879   0.000000
2      2012           3        0.000000          3.688879   0.000000
3      2012           3        0.000000          3.688879   3.931826
4      2012           3        0.000000          3.688879   3.931826
5      2012           3        0.000000          3.688879   3.931826
6      2012           3        0.000000          3.688879   4.143135
7      2012           3        0.000000          3.688879   4.143135
8      2012           3        0.000000          3.688879   4.143135
9      2012           3        0.000000          3.688879   4.442651
10     2012           3        0.000000          3.688879   4.442651
11     2012           3        0.000000          3.688879   4.442651
12     2012           3        0.000000          3.688879   4.465908
13     2012           3        0.000000          3.688879   4.465908
14     2012           3        0.000000          3.688879   4.465908
15     2012           3        0.000000          3.688879   4.510860
16     2012           3        0.000000          3.688879   4.653960
17     2012           3        0.000000          3.688879   4.653960
18     2012           3        0.000000          3.688879   4.653960
19     2012           3        0.000000          3.688879   4.779123
20     2012           3        0.000000          3.688879   4.779123
21     2012           3        0.000000          3.688879   4.779123
22     2012           3        0.000000          3.912023   1.098612
23     2012           3        0.000000          3.912023   1.098612
24     2012           3        0.000000          3.912023   1.098612
25     2012           3        0.000000          3.912023   2.564949
26     2012           3        0.000000          3.912023   2.564949
27     2012           3        0.000000          3.912023   2.564949
28     2012           3        0.000000          3.912023   3.135494
29     2012           3        0.000000          3.912023   3.135494
...     ...         ...             ...               ...        ...
31091  2002           3        4.025352          2.302585   1.098612
31092  2002           3        4.025352          2.302585   1.098612
31093  2002           3        4.025352          2.302585   2.564949
31094  2002           3        4.025352          2.302585   2.564949
31095  2002           3        4.025352          2.302585   2.833213
31096  2002           3        4.025352          2.302585   2.833213
31097  2002           3        4.025352          2.302585   3.367296
31098  2002           3        4.025352          2.302585   3.367296
31099  2002           3        4.025352          2.302585   3.761200
31100  2002           3        4.025352          2.302585   3.761200
31101  2002           3        4.025352          3.688879   0.000000
31102  2002           3        4.025352          3.688879   0.000000
31103  2002           3        4.025352          3.688879   1.945910
31104  2002           3        4.025352          3.688879   1.945910
31105  2002           3        4.025352          3.688879   3.218876
31106  2002           3        4.025352          3.688879   3.218876
31107  2002           3        4.025352          3.688879   3.610918
31108  2002           3        4.025352          3.688879   3.610918
31109  2002           3        4.025352          3.912023   2.197225
31110  2002           3        4.025352          3.912023   2.197225
31111  2002           3        4.025352          3.912023   2.708050
31112  2002           3        4.025352          3.912023   2.708050
31113  2002           3        4.025352          3.912023   3.044522
31114  2002           3        4.025352          3.912023   3.044522
31115  2002           3        4.025352          3.912023   3.433987
31116  2002           3        4.025352          3.912023   3.433987
31117

# Reminders of what Machine Learning can do for the iris dataset
### borrowed from Jake VanderPlas Ch. 5 on Hyperparameters and Model Validation
#I'm just re-typing this as a guide for my applications

In [48]:
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
iris = load_iris()

Xi = iris.data
yi = iris.target

print(iris) 

{'data': array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
     

In [49]:
print(Xi.shape)
print(yi.shape)

(150, 4)
(150,)


In [50]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)

model.fit(Xi, yi)
yi_model = model.predict(Xi)

from sklearn.metrics import accuracy_score
accuracy_score(yi, yi_model)

1.0

In [51]:
# make holdout sets for training
Xi1, Xi2, yi1, yi2 = train_test_split(Xi, yi, random_state=0, train_size=0.65)

#fit the model
model.fit(Xi1, yi1)

# evaluate the model on the other set
yi2_model = model.predict(Xi2)
accuracy_score(yi2, yi2_model)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


0.9622641509433962

In [52]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, Xi, yi, cv=5)

array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])

# Support Vector Machines : Kernal SVM

In [53]:
from scipy import stats
import seaborn as sns; sns.set()
from sklearn.datasets.samples_generator import make_circles
from sklearn.svm import SVC #'Support Vector Classifier'
X3, y3 = make_circles(100, factor=.1, noise=.1)

clf = SVC(kernel='linear').fit(X3,y3)

plt.scatter(X3[:,0], X3[:,1], c=y3, s=50, cmap='autumn')
model = SVC(kernel='linear', C=1E10)
def plot_svc_decision_function(model, ax=None, plot_support=True):
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0],xlim[1],30)
    
plot_svc_decision_function(clf, plot_support=False);

In [54]:
r = np.exp(-(X3 ** 2).sum(1))
from mpl_toolkits import mplot3d
from ipywidgets import interact, fixed

def plot_3D(elev=30, azim=30, X3=X3, y3=y3):
    ax = plt.subplot(projection='3d')
    ax.scatter3D(X3[:,0], X3[:,1], r, c=y3, s=50, cmap='autumn')
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel('x')
    ax.set_ylabel('y3')
    ax.set_zlabel('r')

interact(plot_3D, elev=[-90,90], azip=(-180, 180),
        X3=fixed(X3), y3=fixed(y3));

interactive(children=(Dropdown(description='elev', options=(-90, 90), value=-90), IntSlider(value=30, descript…

# Dimensionality Reduction

In [66]:
# Try an unsupervised learning technique (Principal Component Analysis) to see, by component, the variance of the components that explains the majority of the data

from sklearn.decomposition import PCA

pca = PCA()
f2_drop_pca = pca.fit_transform(f2_drop)

print(f2_drop_pca)

[[-4.06666662  4.17203051  2.92118544 -0.75862224 -0.18540205]
 [-4.06666662  4.17203051  2.92118544 -0.75862224 -0.18540205]
 [-4.06666662  4.17203051  2.92118544 -0.75862224 -0.18540205]
 ...
 [ 5.94005788  0.87563173 -0.83096648  0.23813068  0.0257864 ]
 [ 5.93907382  0.30267531 -0.79501492  0.30570324  0.03173678]
 [ 5.93907382  0.30267531 -0.79501492  0.30570324  0.03173678]]


In [67]:
# Try again using the fe_data look at accuracy of k-nearest neighbors (knn) model on the set using the normalized Year

# Create a data with all columns except 'Year'
yr_X = f2_drop.drop('Year', axis=1)

# Create a 'Year' labels dataset
yr_y = f2_drop[['Year']]

# Use stratified sampling to split up the dataset according to the yr_y dataset
X_train, X_test, y_train, y_test = train_test_split(yr_X, yr_y, stratify=yr_y)

# Fit the k-nearest neighbors model to the training data
knn.fit(X_train, y_train)

# Score the model on the test data
print(knn.score(X_test, y_test))


# Print out the category_desc counts on the training y labels
print(y_train['Year'].value_counts())
print('\n', y_train.head())

0.272068511198946
2012    10243
2007     6545
2002     5981
Name: Year, dtype: int64

        Year
16775  2007
26823  2002
17242  2007
5147   2012
14358  2007


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  del sys.path[0]


In [72]:
# attempt to fix the above error using the suggested np.ravel()
print(f2_drop.shape())


TypeError: 'tuple' object is not callable